# Aliasing

$$
\require{cancel}\notag%
\require{color}\notag%
$$

The previous section introduced uniform sampling, which allows us to represent a continuous signal $x(t)$ by a discrete sequence of sample values $x[n]$.

In this section, we'll see that this idea comes with some restrictions.


In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cycler
import matplotlib.path as path
import matplotlib.patches as patches
from matplotlib.transforms import Bbox
import warnings
warnings.simplefilter('ignore')

from matplotlib.animation import FuncAnimation
import matplotlib.style
matplotlib.style.use('bauhaus')

np.random.seed(20200101)

colors = [_['color'] for _ in list(matplotlib.rcParams['axes.prop_cycle'])]

def figaspect(arg):
    # min/max sizes to respect when autoscaling.  If John likes the idea, they
    # could become rc parameters, for now they're hardwired.
    figsize_min = np.array((4.0, 2.0))  # min length for width/height
    figsize_max = np.array((16.0, 16.0))  # max length for width/height

    # Extract the aspect ratio of the array
    isarray = hasattr(arg, 'shape') and not np.isscalar(arg)
    if isarray:
        nr, nc = arg.shape[:2]
        arr_ratio = nr / nc
    else:
        arr_ratio = arg

    # Height of user figure defaults
    fig_width = matplotlib.rcParams['figure.figsize'][0]

    # New size for the figure, keeping the aspect ratio of the caller
    newsize = np.array((fig_width, fig_width *arr_ratio))
    #newsize = np.array((fig_height / arr_ratio, fig_height))

    # Sanity checks, don't drop either dimension below figsize_min
    newsize /= min(1.0, *(newsize / figsize_min))

    # Avoid humongous windows as well
    newsize /= max(1.0, *(newsize / figsize_max))

    # Finally, if we have a really funky aspect ratio, break it but respect
    # the min/max dimensions (we don't want figures 10 feet tall!)
    newsize = np.clip(newsize, figsize_min, figsize_max)
    return newsize

## What is aliasing?

**Aliasing** is the name we give to the phenomenon when two distinct continuous signals $x_1(t)$ and $x_2(t)$ produce the same sequence of sample values $x[n]$ when sampled at a fixed rate $f_s$.

More specifically, we usually think of aliasing in terms of pure (sinusoidal) tones $x(t) = A \cdot \cos\left(2\pi \cdot f \cdot t + \phi\right)$.
Given a sampling rate $f_s$, two frequencies $f$ and $f'$ are **aliases** of each other if for some integer $k$,

$$
f' = f + k \cdot f_s.
$$

Or, in words, frequency $f'$ is $f$ plus some whole number multiples of the sampling rate $f_s$.  This rule for relating aliased frequencies is known as the *aliasing equation*.

The figure below illustrates this effect: for any $f$ that we choose, once a sampling rate $f_s$ is chosen, there are infinitely many frequencies that produce the exact same samples.

```{margin} Aside: aliasing units

The aliasing equation combines a frequency $f$ `[cycles/sec]` with a sampling rate $f_s$ `[samples/sec]`, scaled by an integer $k$.
Following our previous discussion of dimension analysis in [chapter 1](/ch01/Units.html), we'll need to assign some units to $k$ for this to be well-defined.
Since $f'$ should also be in `[cycles/sec]`, we should define $k$ as `[cycles/sample]` so that $k\cdot f_s$ has units `[cycles/sec]` and can be added to $f$.

This is not just being pedantic: giving units to $k$ helps us reason about what it means inuitively.  It measures additional **cycles between samples**!
```

In [14]:
%%capture
## Animation of aliasing
frame_rate = 25

# full wave should stay on screen for 2 seconds
# full wave should take 2 seconds to wipe out
# one second between waves
# each wave = 7 seconds * 4 waves = 28 seconds of animation

anim_duration = 28
n_frames = anim_duration * frame_rate

sched_0 = np.zeros(n_frames) + 3
sched_0[0*frame_rate:2*frame_rate] = np.linspace(-2, 0, endpoint=False, num=2*frame_rate)
sched_0[2*frame_rate:4*frame_rate] = 0
sched_0[4*frame_rate:6*frame_rate] = np.linspace(0, 2, endpoint=False, num=2*frame_rate)

sched_1 = np.zeros(n_frames) + 3
sched_1[7*frame_rate:9*frame_rate] = np.linspace(-2, 0, endpoint=False, num=2*frame_rate)
sched_1[9*frame_rate:11*frame_rate] = 0
sched_1[11*frame_rate:13*frame_rate] = np.linspace(0, 2, endpoint=False, num=2*frame_rate)

sched_2 = np.zeros(n_frames) + 3
sched_2[14*frame_rate:16*frame_rate] = np.linspace(-2, 0, endpoint=False, num=2*frame_rate)
sched_2[16*frame_rate:18*frame_rate] = 0
sched_2[18*frame_rate:20*frame_rate] = np.linspace(0, 2, endpoint=False, num=2*frame_rate)

sched_3 = np.zeros(n_frames) + 3
sched_3[21*frame_rate:23*frame_rate] = np.linspace(-2, 0, endpoint=False, num=2*frame_rate)
sched_3[23*frame_rate:25*frame_rate] = 0
sched_3[25*frame_rate:27*frame_rate] = np.linspace(0, 2, endpoint=False, num=2*frame_rate)

# --- #

fig, ax = plt.subplots(figsize=figaspect(0.5))

fs_real = 1000
fs = 5
duration = 2

f0 = 1

f1 = f0 + fs
f2 = f0 + 2 * fs
f3 = f0 - fs

# Plot the continuous time curves
t = np.linspace(0, duration, num=duration * fs_real, endpoint=False)
x0 = np.cos(2 * np.pi * f0 * t)
x1 = np.cos(2 * np.pi * f1 * t)
x2 = np.cos(2 * np.pi * f2 * t)
x3 = np.cos(2 * np.pi * f3 * t)

# Plot the samples
N = int(duration * fs)
xsamp = np.cos(2 * np.pi * f0 * np.arange(N) / fs)
tsamp = np.arange(N) / fs

ax.plot(tsamp, xsamp, linestyle='', marker='.', color=colors[1], label=r'$x[n]$', zorder=5)

p0 = ax.plot([], [], label=r'$f={}$ Hz'.format(f0), color=colors[0])[0]
p1 = ax.plot([], [], label=r'$f={}$ Hz'.format(f1), color=colors[2])[0]
p2 = ax.plot([], [], label=r'$f={}$ Hz'.format(f2), color=colors[3])[0]
p3 = ax.plot([], [], label=r'$f={}$ Hz'.format(f3), color=colors[4])[0]


ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1))

ax.set(title=r'Aliasing example at $f_s={}$ Hz'.format(fs),
        xlabel='Time $t$ [seconds]',
        ylim=[-1.25, 1.25],
        xlim=[0, 2.0],
        yticks=np.linspace(-1, 1, num=9))

def init():
    return p0, p1, p2, p3

def animate(i):
    time = i / float(frame_rate)
    
    if time <= 7:
        n = (t >= sched_0[i]) & (t <= sched_0[i] + 2)
        p0.set_data(t[n], x0[n])
        return (p0,)
    elif time <= 14:
        n = (t >= sched_1[i]) & (t <= sched_1[i] + 2)
        p1.set_data(t[n], x1[n])
        return (p1,)
    elif time <= 21:
        n = (t >= sched_2[i]) & (t <= sched_2[i] + 2)
        p2.set_data(t[n], x2[n])
        return (p2,)
    else:
        n = (t >= sched_3[i]) & (t <= sched_3[i] + 2)
        p3.set_data(t[n], x3[n])
        return (p3,)

animation = FuncAnimation(fig, animate,
                               init_func=init,
                               frames=n_frames,
                               interval=1000./frame_rate,
                               blit=True)

In [15]:
animation

## Why is aliasing a problem?

Aliasing is an unavoidable consequence of digital sampling: there will always be frequencies that look the same after sampling.
The consequence of this fact is that once you've sampled a signal, you may not be able to determine the frequency of the wave that produced the samples you've observed.

We'll see in the next section that the Nyquist-Shannon theorem suggests a resolution to this problem, but for now let's work toward a better understanding of why aliasing occurs.

```{margin} Negative frequency?
You may have noticed that we can have $k < 0$ in the aliasing equation, which can produce frequencies like $f=-4$ Hz.
What does this mean?

You can think of "negative frequency" as the frequency of a point traveling clockwise around the circle, rather than counter-clockwise.
However, if you watch the animation carefully, you'll notice that the curve for $f=-4$ looks just like a wave at $f=+4$, at least for cosine waves.
(For sine waves, the picture is almost the same, but one of the waves would have to flip upside-down.)

We'll see in the next section why this idea is so important.
```

## Why does aliasing happen?

In the previous section, we sampled a pure tone by using the following equation

$$
x[n] = \cos\left(2\pi \cdot f \cdot \frac{n}{f_s} \right).
$$

To see why aliasing occurs between frequencies $f$ and $f + \color{#EC2830}{k \cdot f_s}$, we can plug the latter into the equation above and see what happens.
The key idea that we'll need is the following identity:

$$
\cos(\theta + 2\pi\cdot m) = \cos(\theta) \;\;\;\text{for any integer } m.
$$

This identity works because if $m$ is an integer, then $2\pi\cdot m$ is a whole number of rotations around the circle (clockwise if $m < 0$, counter-clockwise if $m > 0$).
As a result, adding $2\pi\cdot m$ to any angle $\theta$ leaves you back at $\theta$.

By analogy, you can think of $\theta$ as the minute hand on a clock, and each $m$ counts an hour offset.  Regardless of the current time, adding one hour (or two, or three, counted by $m$) will leave the minute hand in exactly the same place.

### Proof of aliasing

The proof of the aliasing equation uses a bit of algebra to re-arrange the terms of the equation, and then exploits the identiy defined above to simplify the equation.
We start with the definition for the sequence generated by sampling a pure tone at $f + \color{#EC2830}{k\cdot f_s}$, and work our way toward the sequence generated by $f$.
We'll ignore amplitude $A$ and phase offset $\phi$ to avoid cluttering the notation, but the argument goes through just as well when those features are included.

$$
\begin{align*}
\cos\left(2\pi \cdot \left(f + \color{#EC2830}{k\cdot f_s}\right) \cdot \frac{n}{f_s} \right)
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{#EC2830}{2\pi \cdot k\cdot f_s \cdot \frac{n}{f_s}} \right) &\text{Distribute multiplication}\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{#EC2830}{2\pi \cdot k\cdot \cancel{f_s} \cdot \frac{n}{\cancel{f_s}}} \right)&\text{Cancel } \frac{f_s}{f_s}\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{#EC2830}{2\pi \cdot k\cdot n} \right) & k\cdot n \text{ is an integer}\\
&= 
\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}  + \color{#EC2830}{\cancel{2\pi \cdot k\cdot n}} \right)& \text{Cancel extra whole cycles}\\
&=\cos\left(2\pi \cdot f \cdot \frac{n}{f_s}\right)\\
&= x[n].
\end{align*}
$$

This shows that the two frequencies $f$ and $f + \color{#EC2830}{k\cdot f_s}$ produce the same sequence of samples, regardless of $k$. (But definitely depending on $f_s$.)

If we were to listen to sampled tones at these frequencies, we shouldn't be able to tell them apart.
Let's test that hypothesis!

## Example: aliased tones

The following code example generates two tones at aliasing frequencies, but is otherwise very similar to the example in the previous section.
Try it out!  The two sequences of samples will be numerically identical, and therefore sound identical.

Here, we just used $k=1$, but any integer $k$ will produce the same results.

In [5]:
import numpy as np
from IPython.display import display, Audio

# Sampling at 8 KHz
fs = 8000

# We'll make a pure tone at 220, and at 220 + 8000 = 8220
f0_original = 220
f0_aliased = f0_original + fs

# 2 seconds of audio should be plenty
duration = 2

# How many samples is 2 seconds?
N = int(duration * fs)

# Generate the sample positions
n = np.arange(N)

# Construct the first signal
x_original = np.cos(2 * np.pi * f0_original * n / fs)

# And the aliased signal
x_aliased = np.cos(2 * np.pi * f0_aliased * n / fs)

# Let's hear them both
print("fs = {} Hz, f0 = {} Hz".format(fs, f0_original))
display(Audio(data=x_original, rate=fs))

print("fs = {} Hz, f0 aliased = {} Hz".format(fs, f0_aliased))
display(Audio(data=x_aliased, rate=fs))

fs = 8000 Hz, f0 = 220 Hz


fs = 8000 Hz, f0 aliased = 8220 Hz
